# Web-Scraping News from World Health Organization
We will scrape the latest News-posts from WHO and compile them to a Dataframe that can be exported to different file-types.

In [1]:
from bs4 import BeautifulSoup # The nice libraries
import urllib.request as ur, pandas as pd

# tag-tokens per post that we are looking for, unnecessary but kept here just in case
cards_ref = dict(title=['h2',{'class':'blog-shortcode-post-title'}],
                 meta=['p',{'class':'fusion-single-line-meta'}],
                 info=['div',{'class':'fusion-post-content-container'}]
                 )

# Collecting the HTML Cards/Panels per post on the 'News' section of the site
url = 'https://www.un.org/youthenvoy/news/'
req = ur.Request(url, headers = {'User-Agent': 'Someone'})
page_content = ur.urlopen(req).read()
soup = BeautifulSoup(page_content, "html.parser")
cards = soup.find_all("div", attrs={'class':'fusion-post-content post-content'})

print("Page-1 Posts collected (in HTML) from WHO's News-page.")

Page-1 Posts collected (in HTML) from WHO's News-page.


### Collecting details per News-post (title, date, etc...)

In [2]:
print('Details being collected per news-post card, an example of the first 3:\n')

who_news_cards = []
for i, card in enumerate(cards, 1):
    
    card_link = card.find('a', href=True)
    title = card_link.text.strip() # Or: card.find('h2').text
    date = card.find('span').text.strip()
    subtext = card.find('div',
                        attrs={'class':'fusion-post-content-container'}).text.strip()
    card_url = card_link['href']
    
    if i < 4: 
        print(f'Title:\t\t\t{title}\n'+
              f'Date:\t\t\t{date}\n'+
              f'Subtext:\t\t{subtext}...\n'+
              f'Read more (post-link):\t{card_link["href"]}\n\n')
    
    who_news_cards.append([title, date, subtext, card_url])

Details being collected per news-post card, an example of the first 3:

Title:			Launch of “Be Seen, Be Heard” Campaign
Date:			11 May 2022
Subtext:		The United Nations Secretary-General’s Envoy on Youth and The Body Shop launch global collaboration calling for more young voices in the halls of power   The Body Shop and United Nations Secretary-General’s Envoy on Youth...
Read more (post-link):	https://www.un.org/youthenvoy/2022/05/launch-of-the-be-seen-be-heard-campaign/


Title:			World leaders renew commitment to “Youth, Peace and Security” agenda
Date:			21 January 2022
Subtext:		Governments announce new actions to meaningfully include youth in peacebuilding efforts at the High-Level Global Conference on Youth-Inclusive Peace Processes 21 January 2022 (Doha, Qatar) —  At the virtual High-Level Global Conference on Youth-Inclusive...
Read more (post-link):	https://www.un.org/youthenvoy/2022/01/world-leaders-renew-commitment-to-youth-peace-and-security-agenda/


Title:			First-ever G

### News-Posts compiled as a Dataframe

In [3]:
who_news_df = pd.DataFrame(who_news_cards, columns=['WHO_News','Date','Subtext','Link'])

who_news_df

,WHO_News,Date,Subtext,Link
0,"Launch of “Be Seen, Be Heard” Campaign",11 May 2022,The United Nations Secretary-General’s Envoy o...,https://www.un.org/youthenvoy/2022/05/launch-o...
1,"World leaders renew commitment to “Youth, Peac...",21 January 2022,Governments announce new actions to meaningful...,https://www.un.org/youthenvoy/2022/01/world-le...
2,First-ever Global Report on Protecting Young P...,18 June 2021,The launch of the report comes alongside a hig...,https://www.un.org/youthenvoy/2021/06/first-ev...
3,Young Human Rights Defenders Adapting to COVID-19,9 December 2020,Celebrating Youth Resilience & Creativity on I...,https://www.un.org/youthenvoy/2020/12/young-hu...
4,"On International Volunteers Day, Meet Our Firs...",4 December 2020,"5 December 2020 Earlier this year, the Off...",https://www.un.org/youthenvoy/2020/12/on-inter...
5,Breaking gender barriers for young women parli...,25 November 2020,Twenty-five years after the Beijing Declaratio...,https://www.un.org/youthenvoy/2020/11/breaking...
6,Statement Issued by the Office of the UN Secre...,14 April 2020,The Office of the UN Secretary-General’s Envoy...,https://www.un.org/youthenvoy/2020/04/statemen...
7,Want to be a Fellow? Apply Now! No more unpaid...,5 February 2020,The Office of the Secretary-General’s Envoy on...,https://www.un.org/youthenvoy/2020/02/want-to-...
8,UN Secretary-General’s Envoy on Youth visits G...,24 December 2019,"In October 2019, the United Nations Secretary-...",https://www.un.org/youthenvoy/2019/12/un-secre...
9,Summer of Solutions,29 November 2019,As the effects of climate change become more a...,https://www.un.org/youthenvoy/2019/11/summer-o...


Some may prefer a dataframe with dates as our index, lets set it this way with "Dates" as date-time objects:

In [4]:
datetime_index = pd.to_datetime(who_news_df.Date)

who_news_df.set_index(datetime_index, inplace=True)
who_news_df.drop('Date', axis=1, inplace=True)

print("A cleaner dataframe indexed Date-times:")
who_news_df

A cleaner dataframe indexed Date-times:


,WHO_News,Subtext,Link
Date,,,
2022-05-11,"Launch of “Be Seen, Be Heard” Campaign",The United Nations Secretary-General’s Envoy o...,https://www.un.org/youthenvoy/2022/05/launch-o...
2022-01-21,"World leaders renew commitment to “Youth, Peac...",Governments announce new actions to meaningful...,https://www.un.org/youthenvoy/2022/01/world-le...
2021-06-18,First-ever Global Report on Protecting Young P...,The launch of the report comes alongside a hig...,https://www.un.org/youthenvoy/2021/06/first-ev...
2020-12-09,Young Human Rights Defenders Adapting to COVID-19,Celebrating Youth Resilience & Creativity on I...,https://www.un.org/youthenvoy/2020/12/young-hu...
2020-12-04,"On International Volunteers Day, Meet Our Firs...","5 December 2020 Earlier this year, the Off...",https://www.un.org/youthenvoy/2020/12/on-inter...
2020-11-25,Breaking gender barriers for young women parli...,Twenty-five years after the Beijing Declaratio...,https://www.un.org/youthenvoy/2020/11/breaking...
2020-04-14,Statement Issued by the Office of the UN Secre...,The Office of the UN Secretary-General’s Envoy...,https://www.un.org/youthenvoy/2020/04/statemen...
2020-02-05,Want to be a Fellow? Apply Now! No more unpaid...,The Office of the Secretary-General’s Envoy on...,https://www.un.org/youthenvoy/2020/02/want-to-...
2019-12-24,UN Secretary-General’s Envoy on Youth visits G...,"In October 2019, the United Nations Secretary-...",https://www.un.org/youthenvoy/2019/12/un-secre...


Later InshAllah we will collect **all** News-posts from WHO's site! Asalamualaikum, cheers.